# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/weather_df.csv"
weather_dataframe = pd.read_csv(weather_csv_file)
weather_dataframe

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,talnakh,94,RU,1650910725,97,69.4865,88.3972,254.93,1.81
1,1,bluff,30,NZ,1650910709,66,-46.6000,168.3333,283.60,10.48
2,2,saskylakh,83,RU,1650910706,100,71.9167,114.0833,250.63,1.71
3,3,ostrovnoy,100,RU,1650910725,98,68.0531,39.5131,274.33,1.92
4,4,ushuaia,100,AR,1650910699,67,-54.8000,-68.3000,278.94,1.50
...,...,...,...,...,...,...,...,...,...,...
574,574,suntar,73,RU,1650910917,96,62.1444,117.6319,267.68,1.39
575,575,pucallpa,75,PE,1650910917,70,-8.3791,-74.5539,303.17,3.09
576,576,sakakah,45,SA,1650910918,47,29.9697,40.2064,295.91,2.97
577,577,mazarron,9,ES,1650910918,55,37.5992,-1.3149,292.97,2.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_dataframe[["Lat","Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity = weather_dataframe["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit weather conditions.
narrow_dataframe = weather_dataframe.loc[(weather_dataframe["Wind Speed"] < 10) & (weather_dataframe["Cloudiness"] < 10) & 
                                   (weather_dataframe["Max Temp"] >= 275) & (weather_dataframe["Max Temp"] <= 300) & (weather_dataframe["Humidity"] <= 30)]

narrow_dataframe.dropna()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,valentin gomez farias,0,MX,1650910728,12,28.2167,-106.5833,297.25,4.35
58,58,twin falls,0,US,1650910620,25,42.3505,-114.6445,288.34,4.12
140,140,saint george,0,US,1650910768,15,37.1041,-113.5841,295.72,1.54
214,214,guerrero negro,0,MX,1650910791,27,27.9769,-114.0611,299.00,2.57
263,263,san quintin,0,MX,1650910561,27,30.4833,-115.9500,297.12,3.17
506,506,farmington,0,US,1650910789,28,36.7281,-108.2187,288.58,1.79
508,508,mut,2,TR,1650910895,23,36.6439,33.4389,295.65,2.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_dataframe.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,valentin gomez farias,MX,28.2167,-106.5833,
58,twin falls,US,42.3505,-114.6445,
140,saint george,US,37.1041,-113.5841,
214,guerrero negro,MX,27.9769,-114.0611,
263,san quintin,MX,30.4833,-115.9500,
506,farmington,US,36.7281,-108.2187,
508,mut,TR,36.6439,33.4389,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)


Retrieving Results for Index 14: valentin gomez farias.
Missing field/result... skipping.
------------
Retrieving Results for Index 58: twin falls.
Missing field/result... skipping.
------------
Retrieving Results for Index 140: saint george.
Closest hotel in saint george is Hilton Garden Inn St. George.
------------
Retrieving Results for Index 214: guerrero negro.
Closest hotel in guerrero negro is The Halfway Inn.
------------
Retrieving Results for Index 263: san quintin.
Closest hotel in san quintin is Old Mill Hotel.
------------
Retrieving Results for Index 506: farmington.
Closest hotel in farmington is Fairfield Inn & Suites by Marriott Farmington.
------------
Retrieving Results for Index 508: mut.
Closest hotel in mut is Konak Otel.
------------


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,valentin gomez farias,MX,28.2167,-106.5833,
58,twin falls,US,42.3505,-114.6445,
140,saint george,US,37.1041,-113.5841,Hilton Garden Inn St. George
214,guerrero negro,MX,27.9769,-114.0611,The Halfway Inn
263,san quintin,MX,30.4833,-115.9500,Old Mill Hotel
506,farmington,US,36.7281,-108.2187,Fairfield Inn & Suites by Marriott Farmington
508,mut,TR,36.6439,33.4389,Konak Otel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))